# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [1]:
import pandas as pd
import boto3
import sagemaker

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [6]:
# should be the name of directory you created to save your features data
data_dir = "plagiarism_data"

# set prefix, a descriptive name for a directory  
prefix = "project_plagiarism_detection"

# upload all data to S3

s3_data_dir = sagemaker_session.upload_data(data_dir, bucket, prefix)
s3_data_dir

's3://sagemaker-us-east-1-642356378541/project_plagiarism_detection'

### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

project_plagiarism_detection/test.csv
project_plagiarism_detection/train.csv
Test passed!


---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

In [9]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train.py

from __future__ import print_function

import argparse
import os
import pandas as pd

# sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. 
# from sklearn.externals import joblib
# Import joblib package directly
import joblib

## TODO: Import any additional libraries you need to define a model
import sklearn


# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")
    
    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")
    
    return model


## TODO: Complete the main code
if __name__ == '__main__':
    
    # All of the model parameters and training parameters are sent as arguments
    # when this script is executed, during a training job
    
    # Here we set up an argument parser to easily access the parameters
    parser = argparse.ArgumentParser

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [10]:

# your import and estimator code, here
from sagemaker.sklearn.estimator import SKLearn


## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [25]:
s3_train_path = f"{s3_data_dir}/train.csv"
s3_test_path = f"{s3_data_dir}/test.csv"
s3_output_path = f"s3://{bucket}/{prefix}/out"

In [26]:
s3_train_path

's3://sagemaker-us-east-1-642356378541/project_plagiarism_detection/train.csv'

In [27]:
from sagemaker.inputs import TrainingInput
train_channel = TrainingInput(s3_train_path, 
                              content_type="text/csv")

In [28]:
train_channel

In [35]:
%%time

# Train your estimator on S3 training data
estimator = SKLearn(entry_point="train.py", 
                    source_dir="source_sklearn",
                    instance_count=1,
                    instance_type="ml.c4.xlarge",
                    framework_version="0.20.0",
                    py_version="py3",
                    role=role, 
                    sagemaker_session=sagemaker_session,
                    output_path=s3_output_path)

# TODO uncomment
# estimator.set_hyperparameters(n_neighbors=2, knn_weights_type="distance")

estimator.fit(inputs={"train": train_channel})

2021-01-10 12:30:54 Starting - Starting the training job...
2021-01-10 12:31:17 Starting - Launching requested ML instancesProfilerReport-1610281853: InProgress
......
2021-01-10 12:32:18 Starting - Preparing the instances for training......
2021-01-10 12:33:19 Downloading - Downloading input data...
2021-01-10 12:33:55 Training - Training image download completed. Training in progress..2021-01-10 12:33:55,626 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
2021-01-10 12:33:55,628 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-01-10 12:33:55,641 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-01-10 12:33:55,866 botocore.utils INFO     IMDS ENDPOINT: http://169.254.169.254/
2021-01-10 12:33:56,009 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
2021-01-10 12:33:57,253 sagemaker-

In [49]:
%%time

# Second estimator, with hyperparameters that got better accuracy
# while runnin tests locally using sklearn: neighbors=2, weights="distance"

estimator = SKLearn(entry_point="train.py", 
                    source_dir="source_sklearn",
                    instance_count=1,
                    instance_type="ml.c4.xlarge",
                    framework_version="0.20.0",
                    py_version="py3",
                    role=role, 
                    sagemaker_session=sagemaker_session,
                    output_path=s3_output_path,
                    hyperparameters={
                        "n_neighbors": 2, 
                        "knn_weights_type": "distance"
                    })


estimator.fit(inputs={"train": train_channel})

2021-01-10 13:29:39 Starting - Starting the training job...
2021-01-10 13:30:02 Starting - Launching requested ML instancesProfilerReport-1610285379: InProgress
.........
2021-01-10 13:31:23 Starting - Preparing the instances for training......
2021-01-10 13:32:38 Downloading - Downloading input data
2021-01-10 13:32:38 Training - Downloading the training image...
2021-01-10 13:33:05 Uploading - Uploading generated training model2021-01-10 13:32:58,588 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
2021-01-10 13:32:58,590 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-01-10 13:32:58,600 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-01-10 13:32:58,781 botocore.utils INFO     IMDS ENDPOINT: http://169.254.169.254/
2021-01-10 13:32:59,030 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirem

## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [36]:
%%time

# deploy your model to create a predictor
predictor = estimator.deploy(initial_instance_count=1,
                             instance_type="ml.t2.medium")


---------------!CPU times: user 278 ms, sys: 12.6 ms, total: 291 ms
Wall time: 7min 32s


In [51]:
%%time

# deploy your model to create a predictor
predictor2 = estimator2.deploy(initial_instance_count=1,
                               instance_type="ml.t2.medium")


-------------------!CPU times: user 325 ms, sys: 27.9 ms, total: 353 ms
Wall time: 9min 32s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [37]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [43]:
# First: generate predicted, class labels
test_y_preds = predictor.predict(test_x)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [46]:
# Second: calculate the test accuracy
from sklearn.metrics import accuracy_score

def evaluate(y_pred, y_test):
    """Calculate and print info about prediction accuracy,
    by comparing predicted labels with test labels."""
    tp = sum((y_pred == 1) & (y_test == 1))
    tn = sum((y_pred == 0) & (y_test == 0))
    fp = sum((y_pred == 1) & (y_test == 0))
    fn = sum((y_pred == 0) & (y_test == 1))
    print(f"tp: {tp}, tn: {tn}, fp: {fp}, fn: {fn}")
    acc = accuracy_score(y_pred, y_test)
    print(f"accuracy_score: {acc}")

evaluate(test_y_preds, test_y.values)

## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

tp: 14, tn: 10, fp: 0, fn: 1
accuracy_score: 0.96

Predicted class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

In [90]:
test_y_preds_2 = predictor2.predict(test_x)
evaluate(test_y_preds_2, test_y.values)

tp: 15, tn: 10, fp: 0, fn: 0
accuracy_score: 1.0


### Answer 

First model (K Nearest Neighbor with K=5 and weights="uniform") has just one missclassified point: 
a false negative. The second model (with K=2 and weights="distance") has 100% accuracy. Nice :-)


In [85]:
predictor.predict(test_x.loc[12].values.reshape(1,4))

array([0])

In [87]:
predictor2.predict(test_x.loc[12].values.reshape(1,4))

array([1])

Let's take a closer look at that example, which was incorrectly classified by the first model.
It is 13-th example of the test set. By going back to the previous notebook, we can extract its details:

| File | Task | Category |
| ---  | ---  | ---      |
| *g2pB_taskc.txt* | c | **1 (heavy)** |


* Answer text:

>*There are a large number of models used in solving the problem of Information Retrieval and they are all based on one of three mathematical bases: set theory, algebra and probabilistic.  The vector space model is one of these methods, and it is an algebraic model.*
*In the vector space model a document is represented as a vector.  Within this vector, each dimension corresponds to a separate term (where a term is typically a single word, keyword or phrase.)  If the term doesn’t occur within the document, the value in the vector is zero.  If a term occurs in the document, its value is non-zero.*
*To calculate how relevant each document is in a keyword search the cosine value of the angle between the vectors is easier to calculate instead of the actual angle.*
*The vector space model, however, is not without its limitations: they have small similarity values, long documents are poorly represented; the order of words does not matter; false positive matches may be brought about by terms contained within words themselves; and documents that should match but use different semantics may return false negative matches.   There are a number of other models that are based on or extend the vector space model, and these are designed to try to eradicate these problems.*

---

* The original text of the corresponding task C is stored in **orig_taskc.txt**.

>*Vector space model (or term vector model) is an algebraic model for representing text documents (and any objects, in general) as vectors of identifiers, such as, for example, index terms. It is used in information filtering, information retrieval, indexing and relevancy rankings. Its first use was in the SMART Information Retrieval System.
A document is represented as a vector. Each dimension corresponds to a separate term. If a term occurs in the document, its value in the vector is non-zero. Several different ways of computing these values, also known as (term) weights, have been developed. One of the best known schemes is tf-idf weighting (see the example below).
The definition of term depends on the application. Typically terms are single words, keywords, or longer phrases. If the words are chosen to be the terms, the dimensionality of the vector is the number of words in the vocabulary (the number of distinct words occurring in the corpus).
The vector space model has the following limitations:*
>   1. *Long documents are poorly represented because they have poor similarity values (a small scalar product and a large dimensionality)*
>   2. *Search keywords must precisely match document terms; word substrings might result in a "false positive match"*
>   3. *Semantic sensitivity; documents with similar context but different term vocabulary won't be associated, resulting in a "false negative match".*
>   4. *The order in which the terms appear in the document is lost in the vector space representation.*
   
---

Texts indeed look pretty much different, which is expected, given its category 1 (heavily-modified).

Let's take a look at its similarity features with the original:

In [92]:
test_x.loc[12]

1    0.619792
2    0.047619
3    0.016129
4    0.341584
Name: 12, dtype: float64

Remember that selected features for our case were: 
* c_1
* c_4
* c_7
* LCS

With very low containment 0.0476 for 4-grams and 0.016 for 7-grams, this example is quite tricky to classify as plagiarism.

However, 1-gram containment of 0.62 and Longest Common Subsequence of 0.34 may become discriminant and help correctly classify this data point, given a model that is robust enough (see question 2 below).

### Question 2: How did you decide on the type of model to use? 

### Answer

#### Framework
Since we have a tiny dataset (70 data points, 4 features), i thought that we may start simple and have a baseline model using `scikit-learn` library. And later on, if needed, improve it using more complex model using `PyTorch`.

The reason I chose `scikit-learn` over `PyTorch` for baseline model is essentially the development speed. With `scikit-learn` minimalistic code it is quite fast to create/train/evaluate the model locally with just several lines of code.
That is what I did in my local IPython environment, after downloading `train.csv` and `test.csv` previously created.



#### Classifier

First, I analyzed the results of [scikit-learn classifiers performance comparison](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html), that you posted above. It appeared to me that the very first classifier, `NearestNeighbors`, is probably the most robust, given different geometries of input of binary classification. So that was my choice of the classifier for my baseline model. Here is functions I defined for training and evaluating the kNN classifier:

```python
def train_knn(x_train, y_train, n):
    clf = KNeighborsClassifier(n_neighbors=n)
    clf.fit(x_train, y_train)
    return clf
    
def evaluate(y_pred, y_test, clf):
    y_pred = clf.predict(x_test)
    tp = sum((y_pred == 1) & (y_test == 1))
    tn = sum((y_pred == 0) & (y_test == 0))
    fp = sum((y_pred == 1) & (y_test == 0))
    fn = sum((y_pred == 0) & (y_test == 1))
    print(f"tp: {tp}, tn: {tn}, fp: {fp}, fn: {fn}")
    
    acc = accuracy_score(y_pred, y_test)
    print(f"accuracy_score: {acc}")
    
    probas = clf.predict_proba(x_test)
    for pred, ground_truth, prob in zip(y_pred, y_test, probas):
        print(f"true value: {ground_truth}, predicted {pred}, prob: {prob}")
```

#### Hyperparameters

The results for the first run of `kNearestNeighbors` classifier gave me 0.96 accuracy on the test set, misclassifying only one example. Honestly speaking, that was hell of a result for a trivial baseline model with no hyperparameters tuning.

But, maybe with some tuning, I could achieve 100% accuracy? Main hyperparameters for `kNearestNeighbors` are `n_neighbors` and `weights`. 

So, i ran my experiments by tuning those parameters:
* `n_neighbors` range between 1 and 7 (befault: 5)
* `weights`
    * `"uniform"` (default);
    * `"distance"`(weights inversely proportional to the distance to points).

Every time I printed all predictions, ground truth labels, and also class probabbility using `.predict_probas()` function, to see the results before they are rounded to the closest class (0/1).

For `weights="uniform"` , the very first `n_neighbors=1` gave 100% accuracy on test set!
But, having the model which classifies data points based on 1 and only 1 closest neighbor didn't seem too wise and robust. However, bigger N gave the same baseline 96% accuracy.

For `weights="distance"`, N=1,2,6 gave accuracy=100% on test set, and N=3,4,5,7 gave default 96%. So, for the final model i decided to choose `n_neighbors=2, weights="distance"` parameters, which look very basic but quite logical:
* model will only analyze *two* closest neighbors for a test point;
* to effectively break ties (both points predicting different classes) inverse weights are used. The closer point will have higher weight, which is super logical.

>So, this model gave me 100% accuracy on both TEST and TRAIN sets, so what more to ask for? I thus didn't even try PyTorch solution, since this very basic `KNearestNeighbors` model with some tuning gave me a PERFECT score. I think it is enough generalizable and will not overfit.

#### Note on binary vs multiclass problem

From the very beginning of the project, I was sure that this is a **multiclass** classification problem more than binary. My initial plan was to create another dataset, the one which keeps **Category** column and uses it is target variable instead of binary `Class`. 

Merging all 3 plagiarism categories in one single positive class seems like a **huge loss of information**, potentially leading to sub-optimal models. 

The very nature of the problem gives a notion of ORDER among classes: from 0 = non plagiarism to 3 = copy-paste plagiarism. It is almost like a regression problem, where we may run a regression and then set a threshold value to transform continuous output into binary output.
Or, running a multiclass classifier, which will output one of classes [0,1,2,3] which we may then transform to [0,1] by aggregating all positive classes 1,2,3 to the single "plagiarism" class.

By the way, the only example that was incorrectly classified by a baseline model, has Category=1 ("heavy"). Which proves once again, that data points of this category are the most difficult to classify. Thus, having a multiclass classifier or a regression+classification pipeline would seem as a more robust model. 

However, since after some tuning I obtained 100% accuracy, I was lazy enough and didn't even try a multiclass approach. But definitely wanted to share this thought with you, *dear reviewer*. Please comment on this. Thank you!

----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [93]:
# uncomment and fill in the line below!
predictor.delete_endpoint()
predictor2.delete_endpoint()


### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [ ]:
# deleting bucket, uncomment lines below

# bucket_to_delete = boto3.resource('s3').Bucket(bucket)
# bucket_to_delete.objects.all().delete()

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!